# SlideRule API: Interactive Widget

SlideRule is an on-demand science data processing service that runs in the cloud and responds to REST API calls to process and return science results.

SlideRule can be accessed by any http client (e.g. curl) by making GET and POST requests to the SlideRule service. For the purposes of this document, all requests to SlideRule will originate from a Python script using Python's requests module.

This notebook uses [Jupyter widgets](https://ipywidgets.readthedocs.io) to set parameters for using the SlideRule API.
Regions of interest for submitting to SlideRule are drawn on a [leaflet](https://ipyleaflet.readthedocs.io) map.  Multiple polygons can be submitted at a given time.

#### Load necessary packages

In [1]:
from sliderule import icesat2, ipysliderule, io
import ipywidgets as widgets
import logging
import warnings
# autoreload
%load_ext autoreload
%autoreload 2
# turn off warnings for demo
warnings.filterwarnings('ignore')

#### Set options for making science data processing requests to SlideRule

In [2]:
# set the url for the sliderule service
# set the logging level
icesat2.init("icesat2sliderule.org", loglevel=logging.WARNING)

In [3]:
# create global variables
gdf = None
parms = None

In [4]:
# display widgets for setting SlideRule parameters
SRwidgets = ipysliderule.widgets()
widgets.VBox([
    SRwidgets.asset,
    SRwidgets.release,
    SRwidgets.surface_type,
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.confidence,
    SRwidgets.land_class,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma,
    SRwidgets.projection
])

#### Select regions of interest for submitting to SlideRule

In [5]:
# create ipyleaflet map in specified projection
m = ipysliderule.leaflet(SRwidgets.projection.value)
m.map

Map(center=[39, -108], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

#### Build and transmit requests to SlideRule

In [6]:
def runSlideRule():
    global gdf, parms
    
    # sliderule asset and data release
    asset = SRwidgets.asset.value
    release = SRwidgets.release.value

    # build sliderule parameters using latest values from widget
    parms = {
        # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
        "srt": SRwidgets.surface_type.index,
        # length of ATL06-SR segment in meters
        "len": SRwidgets.length.value,
        # step distance for successive ATL06-SR segments in meters
        "res": SRwidgets.step.value,
        # confidence level for PE selection
        "cnf": SRwidgets.confidence.value,
        # ATL08 land surface classifications
        "atl08_class": list(SRwidgets.land_class.value),
        # maximum iterations, not including initial least-squares-fit selection
        "maxi": SRwidgets.iteration.value,
        # minimum along track spread
        "ats": SRwidgets.spread.value,
        # minimum PE count
        "cnt": SRwidgets.count.value,
        # minimum height of PE window in meters
        "H_min_win": SRwidgets.window.value,
        # maximum robust dispersion in meters
        "sigma_r_max": SRwidgets.sigma.value
    }

    # clear existing geodataframe
    gdf = icesat2.__emptyframe()

    # for each region of interest
    for poly in m.regions:
        # add polygon from map to sliderule parameters
        parms["poly"] = poly 
        # make the request to the SlideRule (ATL06-SR) endpoint
        # and pass it the request parameters to request ATL06 Data
        gdf = gdf.append(icesat2.atl06p(parms, asset, version=release))

    return gdf

#### Add GeoDataFrame to Map
Will limit the plot to be up to 10000 points as a default

In [6]:
def on_button_clicked1(b):
    with output1:
        print("SlideRule processing request...", end="")
        gdf = runSlideRule()
        print(f'complete; returned {gdf.shape[0]} records')
        m.GeoData(gdf, column_name='h_mean', cmap='viridis')

button1 = widgets.Button(description="Run SlideRule!")
output1 = widgets.Output()
button1.on_click(on_button_clicked1)
display(button1, output1)

Button(description='Run SlideRule!', style=ButtonStyle())

Output()

#### Review output
See http://icesat2sliderule.org/rtd/user_guide/ICESat-2.html#elevations for descriptions of each column

In [14]:
def on_button_clicked2(b):
    global gdf
    with output2:
        display(gdf.head())
button2 = widgets.Button(description="Display GeoDataFrame")
output2 = widgets.Output()
button2.on_click(on_button_clicked2)
display(button2, output2)

Button(description='Display GeoDataFrame', style=ButtonStyle())

Output()

#### Save GeoDataFrame to output tile
- [pytables HDF5](https://www.pytables.org/): easily read back as a Geopandas GeoDataFrame
- [netCDF](https://www.unidata.ucar.edu/software/netcdf): interoperable with other programs

In [15]:
display(SRwidgets.filesaver)

In [16]:
def on_button_clicked3(b):
    global gdf
    with output3:
        # append sliderule api version to attributes
        version = icesat2.get_version()
        parms['version'] = version['icesat2']['version']
        parms['commit'] = version['icesat2']['commit']
        # save to file in format (HDF5 or netCDF)
        io.to_file(gdf, SRwidgets.file,
            format=SRwidgets.format,
            driver='pytables',
            parameters=parms,
            regions=m.regions,
            verbose=True)
button3 = widgets.Button(description="Write to File")
output3 = widgets.Output()
button3.on_click(on_button_clicked3)
display(button3, output3)

Button(description='Write to File', style=ButtonStyle())

Output()

#### Read GeoDataFrame from input file
- [pytables HDF5](https://www.pytables.org/)
- [netCDF](https://www.unidata.ucar.edu/software/netcdf)

In [ ]:
display(SRwidgets.fileloader)

In [17]:
def on_button_clicked4(b):
    global gdf
    with output4:
        # read from file in format (HDF5 or netCDF)
        gdf = io.from_file(SRwidgets.file,
            format=SRwidgets.format,
            driver='pytables')
        display(gdf.head())
button4 = widgets.Button(description="Read from File")
output4 = widgets.Output()
button4.on_click(on_button_clicked4)
display(button4, output4)

Button(description='Read from File', style=ButtonStyle())

Output()